In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# Data scraping
import codecs
import requests
from ast import literal_eval

# Standard libraries
import pandas as pd
import numpy as np

# Data processing
import json
from dateutil.parser import parse
import re

# Machine learning
from sklearn.cross_validation import StratifiedKFold
from xgboost import XGBClassifier, XGBRegressor

# Plotting
import matplotlib.pyplot as plt
import matplotlib
from sklearn.metrics import confusion_matrix
from xgboost import plot_importance
import seaborn as sns

# Distance feature extraction
from sklearn.decomposition import PCA
from scipy.spatial import cKDTree

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
_columns = ['id', 'querytime', 'seconds_since_midnight', 'hour', 'weekday', 'month', 'connection', 
            'from', 'from_string', 'from_lat', 'from_lng', 'morning_jam', 'evening_jam',
            'to', 'to_string', 'to_lat', 'to_lng', 'vehicle', 'vehicle_type', 'occupancy',
            'year', 'day', 'quarter', 'vehicle_nr', 'line_nr' ,'from_stop_time', 'to_stop_time']

stations_df = pd.read_csv('stations.csv')
stations_df['URI'] = stations_df['URI'].apply(lambda x: x.split('/')[-1])
week_day_mapping = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}

def get_line_number(vehicle):
    pattern = re.compile("^([A-Z]+)[0-9]+$")
    vehicle_type = pattern.match(vehicle).group(1)
    pattern = re.compile("^[A-Z]+([0-9]+)$")
    vehicle_nr = int(pattern.match(vehicle).group(1))
    line_nr = 0
    if vehicle_type == 'IC':
        line_nr = str(int(100 * np.floor(vehicle_nr / 100)))
    elif vehicle_type == 'L':
        line_nr = str(int(50 * np.floor(vehicle_nr / 50)))
    elif vehicle_type == 'S':
        line_nr = str(int(50 * np.floor(vehicle_nr / 50)))
    else:
        line_nr = 'P'
    
    return vehicle_nr, line_nr

def parse_file(path):
    parsed_logs = []
    faulty_logs = 0
    time_zones = []
    with open(path) as data_file:  
        for line in data_file:
            occ_logline = json.loads(line)
            morning_commute = 0
            evening_commute = 0
            commute_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
            # Do a check if the querytype is occupancy
            if occ_logline['querytype'] == 'occupancy' and 'error' not in occ_logline \
            and 'querytime' in occ_logline:

                    try:
                        query_time = occ_logline['querytime']
                        try:
                            parsed_query_time = parse(query_time)
                            week_day = week_day_mapping[parsed_query_time.weekday()]
                            weekday_nr = parsed_query_time.weekday()
                            midnight = parsed_query_time.replace(hour=0, minute=0, second=0, microsecond=0)
                            seconds_since_midnight = (parsed_query_time - midnight).seconds
                            day = parsed_query_time.day
                            year = parsed_query_time.year
                            month = parsed_query_time.month
                            hour = parsed_query_time.hour
                            quarter = int(parsed_query_time.minute/15)
                            timezone_offset = parsed_query_time.tzinfo._offset
                            time_zones.append(timezone_offset)
                            hours_offset, remainder = divmod(timezone_offset.seconds, 3600)
                            # De ochtendspits valt doorgaans in de periode van 7.00 tot 9.00 uur. 
                            # De avondspits valt in de regel tussen 16.30 en 18.30 uur.
                            if 6 < (hour - hours_offset + 1) < 10 and week_day in commute_days:
                                morning_commute = 1
                            if 15 < (hour - hours_offset + 1) < 19 and week_day in commute_days:
                                evening_commute = 1
                        except ValueError:
                            faulty_logs += 1
                            continue

                        vehicle_id = occ_logline['post']['vehicle'].split('/')[-1]
                        connection = occ_logline['post']['connection']
                        if 'occupancy' in occ_logline['post']:
                            occupancy = occ_logline['post']['occupancy'].split('/')[-1]
                        else:
                            occupancy = 'unknown'
                        
                        if 'to' in occ_logline['post']:
                            to_id = occ_logline['post']['to'].split('/')[-1]
                            if to_id[:2] == '00' and to_id != '00':
                                to_string = stations_df[stations_df['URI'] == to_id]['name'].values[0]
                                to_lat = stations_df[stations_df['URI'] == to_id]['latitude'].values[0]
                                to_lng = stations_df[stations_df['URI'] == to_id]['longitude'].values[0]
                                to_stop_time = stations_df[stations_df['URI'] == to_id]['avg_stop_times'].values[0]
                            else:
                                to_string = None
                                to_lat = None
                                to_lng = None
                                to_stop_time = None
                        else:
                            to_id = None
                            to_string = None
                            to_lat = None
                            to_lng = None
                            to_stop_time = None
                          
                        if 'from' in occ_logline['post']:
                            from_id = occ_logline['post']['from'].split('/')[-1]
                            if from_id[:2] == '00' and from_id != '00':
                                from_string = stations_df[stations_df['URI'] == from_id]['name'].values[0]
                                from_lng = stations_df[stations_df['URI'] == from_id]['longitude'].values[0]
                                from_lat = stations_df[stations_df['URI'] == from_id]['latitude'].values[0]
                                from_stop_time = stations_df[stations_df['URI'] == from_id]['avg_stop_times'].values[0]
                            else:
                                from_string = None
                                from_lat = None
                                from_lng = None
                                from_stop_time = None
                        else:
                            from_id = None
                            from_string = None
                            from_lat = None
                            from_lng = None
                            from_stop_time = None

                        pattern = re.compile("^([A-Z]+)[0-9]+$")
                        try:
                            vehicle_type = pattern.match(vehicle_id).group(1)
                            vehicle_nr, line_nr = get_line_number(vehicle_id)
                        except Exception as e:
                            vehicle_type = None
                            vehicle_nr, line_nr = None, None
                            
                        if 'id' in occ_logline:
                            _id = occ_logline['id']
                        else:
                            _id = -1
                            
                        parsed_logs.append([_id, parsed_query_time, seconds_since_midnight, hour, week_day, month,
                                            connection, from_id, from_string, from_lat, from_lng, morning_commute,
                                            evening_commute, to_id, to_string, to_lat, to_lng, vehicle_id, 
                                            vehicle_type, occupancy, year, day, quarter, vehicle_nr, line_nr,
                                            from_stop_time, to_stop_time])

                    except Exception as e:
                        faulty_logs += 1
                        raise
                        continue
        return parsed_logs, faulty_logs
                    
parsed_file1, faulty1 = parse_file('train.nldjson')
parsed_file2, faulty2  = parse_file('test.nldjson')
logs_df = pd.DataFrame(parsed_file1+parsed_file2)
logs_df.columns = _columns
# old_length = len(logs_df)
# logs_df = logs_df.drop_duplicates(subset=['day', 'month', 'from', 'vehicle'])
# print(len(logs_df) - old_length, 'duplicates removed')

In [3]:
weather_df = pd.concat([pd.read_csv('weather_data_july_1.csv'), 
                        pd.read_csv('weather_data_july_2.csv'), pd.read_csv('weather_data_aug_1.csv'),
                        pd.read_csv('weather_data_aug_2.csv'), pd.read_csv('weather_data_sep_1.csv'),
                        pd.read_csv('weather_data_sep_2.csv'), pd.read_csv('weather_data_oct_1.csv'),
                        pd.read_csv('weather_data_oct_2.csv'), pd.read_csv('weather_data_nov_1.csv'),
                        pd.read_csv('weather_data_nov_2.csv'), pd.read_csv('weather_data_dec_1.csv'),
                        pd.read_csv('weather_data_dec_2.csv')])

weather_features = []
_columns = list(logs_df.columns) + ['temperature_from', 'humidity_from', 'windspeed_from', 
                                    'visibility_from', 'weather_type_from', 'temperature_to', 
                                    'humidity_to', 'windspeed_to', 'visibility_to', 'weather_type_to']

for i in range(len(logs_df)):
    print(i,'/',len(logs_df))
    feature_vector = logs_df.iloc[i,:]
    if feature_vector['querytime'].minute < 30:
        feature_vector_hour = feature_vector['querytime'].hour
    else:
        feature_vector_hour = feature_vector['querytime'].hour + 1
        
    month_string = str(feature_vector['querytime'].month) if feature_vector['querytime'].month > 9 else '0'+str(feature_vector['querytime'].month)
    day_string = str(feature_vector['querytime'].day) if feature_vector['querytime'].day > 9 else '0'+str(feature_vector['querytime'].day)
    hour_string = str(feature_vector_hour%24) if feature_vector_hour%24 > 9 else '0'+str(feature_vector_hour%24)
    feature_vector_date_string = str(feature_vector['querytime'].year)+'-'+month_string+'-'+day_string+' '+hour_string+':00:00'

    weather_entry = weather_df[(weather_df['date_time'] == feature_vector_date_string)
                               & (weather_df['station_name'] == feature_vector['from_string'])].head(1)
    if feature_vector['from_string'] == 'Aalst' or feature_vector['from_string'] == 'Lokeren':
        # Something went wrong for Aalst & Lokeren weather data, let's just take the data from Gent
        weather_entry = weather_df[(weather_df['date_time'] == feature_vector_date_string)
                               & (weather_df['station_name'] == 'Gent-Sint-Pieters')].head(1)
    if feature_vector['from_string'] == 'Mol' or feature_vector['from_string'] == 'Lommel':
        weather_entry = weather_df[(weather_df['date_time'] == feature_vector_date_string)
                               & (weather_df['station_name'] == 'Hasselt')].head(1)    
        
    if len(weather_entry) > 0:
        print('Weather data for (FROM) '+feature_vector['from_string']+' found')
        weather_feature_v = list(feature_vector.values) + [weather_entry['temperature'].values[0], weather_entry['humidity'].values[0], 
                                                            weather_entry['windspeed'].values[0], weather_entry['visibility'].values[0],
                                                            weather_entry['weather_type'].values[0]]
    else:
        weather_feature_v = list(feature_vector.values) + [None, None, None, None, None]
        
        
    weather_entry = weather_df[(weather_df['date_time'] == feature_vector_date_string)
                               & (weather_df['station_name'] == feature_vector['to_string'])].head(1)
    if feature_vector['to_string'] == 'Aalst' or feature_vector['to_string'] == 'Lokeren':
        # Something went wrong for Aalst & Lokeren weather data, let's just take the data from Gent
        weather_entry = weather_df[(weather_df['date_time'] == feature_vector_date_string)
                               & (weather_df['station_name'] == 'Gent-Sint-Pieters')].head(1)
        
    if feature_vector['to_string'] == 'Mol' or feature_vector['to_string'] == 'Lommel':
        weather_entry = weather_df[(weather_df['date_time'] == feature_vector_date_string)
                               & (weather_df['station_name'] == 'Hasselt')].head(1)
    if len(weather_entry) > 0:
        print('Weather data for (TO) '+feature_vector['to_string']+' found')
        weather_feature_v = weather_feature_v + [weather_entry['temperature'].values[0], weather_entry['humidity'].values[0], 
                                                            weather_entry['windspeed'].values[0], weather_entry['visibility'].values[0],
                                                            weather_entry['weather_type'].values[0]]
    else:
        weather_feature_v = weather_feature_v + [None, None, None, None, None]
        
    weather_features.append(weather_feature_v)
    
weather_features_df = pd.DataFrame(weather_features, columns=_columns)

0 / 3779
Weather data for (FROM) Brussel-Centraal/Bruxelles-Central found
1 / 3779
Weather data for (FROM) Brussel-Centraal/Bruxelles-Central found
2 / 3779
Weather data for (FROM) Brussel-Centraal/Bruxelles-Central found
3 / 3779
Weather data for (FROM) Brussel-Centraal/Bruxelles-Central found
4 / 3779
Weather data for (FROM) Brussel-Centraal/Bruxelles-Central found
5 / 3779
Weather data for (FROM) Brussel-Centraal/Bruxelles-Central found
6 / 3779
Weather data for (FROM) Brussel-Centraal/Bruxelles-Central found
7 / 3779
Weather data for (FROM) Brussel-Centraal/Bruxelles-Central found
8 / 3779
Weather data for (FROM) Brussel-Centraal/Bruxelles-Central found
9 / 3779
Weather data for (FROM) Brussel-Centraal/Bruxelles-Central found
10 / 3779
Weather data for (FROM) Brussel-Centraal/Bruxelles-Central found
11 / 3779
Weather data for (FROM) Brussel-Centraal/Bruxelles-Central found
12 / 3779
Weather data for (FROM) Brussel-Centraal/Bruxelles-Central found
13 / 3779
Weather data for (FROM) B

In [56]:
connection_matrix = np.zeros((len(weather_features_df), len(stations_df)))   # Init our sparse matrix
stations_idx = {station: i for i, station in enumerate(stations_df['URI'])}
stations_reverse_idx = {i: station for i, station in enumerate(stations_df['URI'])}
line_information_df = pd.read_csv('line_info.csv')
for i in range(len(weather_features_df)):
    logs_entry = weather_features_df.iloc[i, :]
    logs_entry_from = logs_entry['from']
    if logs_entry_from == '008891173': logs_entry_from = '008891553'  # Some manual fixing (Zjeebruhhe)
    #if logs_entry_from == '008831401': logs_entry_from = '008833001'  # Some manual fixing
    # Get the line information for this sample
    filtered_line_info_df = line_information_df[line_information_df['vehicle_id'] == logs_entry['vehicle']]
    if len(filtered_line_info_df) > 0:
        line_info_entry = filtered_line_info_df.iloc[0, :]
        vehicle_type, stop_station_ids = line_info_entry['vehicle_type'], literal_eval(line_info_entry['stopping_station_ids'])
        #print(stop_station_ids, logs_entry['from'], logs_entry['vehicle'])
        try:
            current_station_idx = stop_station_ids.index(logs_entry_from)
            for j in range(len(stop_station_ids)):
                connection_matrix[i, stations_idx[stop_station_ids[j]]] = j - current_station_idx - (1 * (j - current_station_idx <= 0))
        except:
            print(logs_entry['from'], stop_station_ids)

008200518 ['008400526', '008821402', '008821436', '008821444', '008821451', '008821519', '008821535', '008821543', '008821071', '008821089', '008821063', '008821006', '008821121', '008821196', '008824158', '008824224', '008824232', '008824240', '008822814', '008822848', '008822715']
008872009 ['008814001', '008813003', '008812005', '008811189', '008822004', '008822343', '008821238', '008821121', '008821006']
008813003 ['008819406', '008811163', '008811916', '008811304', '008811411', '008811601', '008861200', '008863008', '008863115', '008863156', '008863560', '008863545', '008863503']
008831005 ['008832664', '008832615', '008832573', '008832565', '008832409', '008832433', '008832458', '008821717', '008821600', '008821121', '008821006']
008841004 ['008833001', '008833233', '008833209', '008821832', '008821600', '008821121', '008821006']
008883212 ['008814001', '008813003', '008812005', '008811189', '008822004', '008822343', '008821238', '008821121', '008821006', '008821071', '008821535'

In [57]:
frequency_mapping = {}
for i in range(connection_matrix.shape[1]):
    frequency_mapping[stations_reverse_idx[i]] = len(connection_matrix[:, i][np.nonzero(connection_matrix[:, i])])

In [65]:
feature_vectors = []
_columns = list(weather_features_df.columns)
print(_columns)
for i in range(len(weather_features_df)):
    sum_freq = 0
    rel_freq = 0
    am_freq = 0
    for j in range(len(connection_matrix[i])):
        if connection_matrix[i, j] != 0:
            sum_freq += frequency_mapping[stations_reverse_idx[j]]
            rel_freq += frequency_mapping[stations_reverse_idx[j]] * 1/connection_matrix[i, j]
            if weather_features_df.iloc[i,:]['hour'] < 12:
                am_freq += rel_freq
            else:
                am_freq -= rel_freq   
    feature_vectors.append(list(weather_features_df.iloc[i,:].values) + [sum_freq, rel_freq, am_freq])
logs_df = pd.DataFrame(feature_vectors)
logs_df.columns = _columns + ['sum_freq', 'rel_freq', 'am_freq']

['id', 'querytime', 'seconds_since_midnight', 'hour', 'weekday', 'month', 'connection', 'from', 'from_string', 'from_lat', 'from_lng', 'morning_jam', 'evening_jam', 'to', 'to_string', 'to_lat', 'to_lng', 'vehicle', 'vehicle_type', 'occupancy', 'year', 'day', 'quarter', 'vehicle_nr', 'line_nr', 'from_stop_time', 'to_stop_time', 'temperature_from', 'humidity_from', 'windspeed_from', 'visibility_from', 'weather_type_from', 'temperature_to', 'humidity_to', 'windspeed_to', 'visibility_to', 'weather_type_to']


In [66]:
def convert_to_am_freq(x):
    return x['rel_freq'] if x['hour'] < 12 else -x['rel_freq']
print(logs_df.columns)   
logs_df['am_freq2'] = logs_df.apply(convert_to_am_freq, axis=1)
print(list(logs_df[logs_df['occupancy'] != 'unknown'].columns))
sns.set()

# sns.pairplot(logs_df[logs_df['occupancy'] != 'unknown'][['occupancy', 'rel_freq', 'sum_freq', 'am_freq', 'am_freq2']],hue="occupancy")

Index(['id', 'querytime', 'seconds_since_midnight', 'hour', 'weekday', 'month',
       'connection', 'from', 'from_string', 'from_lat', 'from_lng',
       'morning_jam', 'evening_jam', 'to', 'to_string', 'to_lat', 'to_lng',
       'vehicle', 'vehicle_type', 'occupancy', 'year', 'day', 'quarter',
       'vehicle_nr', 'line_nr', 'from_stop_time', 'to_stop_time',
       'temperature_from', 'humidity_from', 'windspeed_from',
       'visibility_from', 'weather_type_from', 'temperature_to', 'humidity_to',
       'windspeed_to', 'visibility_to', 'weather_type_to', 'sum_freq',
       'rel_freq', 'am_freq'],
      dtype='object')
['id', 'querytime', 'seconds_since_midnight', 'hour', 'weekday', 'month', 'connection', 'from', 'from_string', 'from_lat', 'from_lng', 'morning_jam', 'evening_jam', 'to', 'to_string', 'to_lat', 'to_lng', 'vehicle', 'vehicle_type', 'occupancy', 'year', 'day', 'quarter', 'vehicle_nr', 'line_nr', 'from_stop_time', 'to_stop_time', 'temperature_from', 'humidity_from', 'wind

In [8]:
print(logs_df[logs_df['vehicle_type'] == 'IC']['sum_freq'].mean())
print(logs_df[logs_df['vehicle_type'] == 'L']['sum_freq'].mean())
print(logs_df[logs_df['vehicle_type'] == 'IC']['rel_freq'].mean())
print(logs_df[logs_df['vehicle_type'] == 'L']['rel_freq'].mean())
print(logs_df[logs_df['vehicle_type'] == 'IC']['am_freq'].mean())
print(logs_df[logs_df['vehicle_type'] == 'L']['am_freq'].mean())
print(logs_df[logs_df['vehicle_type'] == 'IC']['am_freq2'].mean())
print(logs_df[logs_df['vehicle_type'] == 'L']['am_freq2'].mean())

9553.956704497688
2928.9312714776634
141.8182635252264
-143.9441471906685
7681.269105946648
1395.46053559114
644.8629836323588
121.82549747391812


In [67]:
# logs_df = pd.read_csv('kaggle_logs.csv')
logs_df = logs_df[['id', 'seconds_since_midnight', 'weekday', 'vehicle_type', 
                   'month', 'from_lat', 'from_lng', 'to_lat', 'to_lng',
                   'morning_jam', 'evening_jam', 'occupancy', 'hour',
                   'from_stop_time', 'to_stop_time', 'day',
                    'vehicle', 'from', 'rel_freq', 'am_freq', 'am_freq2',
                  'humidity_to', 'humidity_from', 'temperature_to', 'temperature_from']]
# logs_df.to_csv('kaggle_logs.csv')
logs_df['week_day'] = logs_df['weekday']
logs_df = pd.get_dummies(logs_df, columns=['weekday', 'vehicle_type'])

train_df = logs_df[logs_df['occupancy'] != 'unknown']
train_df.to_csv('train.csv')
old_length = len(train_df)
train_df = train_df.drop_duplicates(subset=['day', 'month', 'from', 'vehicle'])
train_df = train_df.dropna()
print(old_length - len(train_df), 'duplicates removed from training set')
print(train_df.head(5))

test_df = logs_df[logs_df['occupancy'] == 'unknown']
test_df.to_csv('test.csv')
print(faulty1+faulty2, 'logs discarded ---', len(logs_df), 'parsed')
print(len(train_df), 'training samples')
print(len(test_df), 'test samples')

293 duplicates removed from training set
    id  seconds_since_midnight  month   from_lat  from_lng     to_lat  \
31  -1                   85306      7  50.859663  4.360846  51.017648   
35  -1                    2308      7  50.896456  4.482076  50.859663   
36  -1                   31827      7  51.035896  3.710675  50.845658   
37  -1                   33350      7  50.872625  4.289895  50.845658   
38  -1                   34562      7  51.035896  3.710675  50.835707   

      to_lng  morning_jam  evening_jam occupancy        ...         \
31  4.482785            0            0      high        ...          
35  4.360846            0            0      high        ...          
36  4.356801            1            0    medium        ...          
37  4.356801            1            0       low        ...          
38  4.336531            1            0    medium        ...          

    vehicle_type_EXT  vehicle_type_IC  vehicle_type_ICE  vehicle_type_ICT  \
31               0.0  

In [68]:
NR_FOLDS = 500
skf = StratifiedKFold(train_df['occupancy'].values, n_folds=10, shuffle=True, random_state=1337)

train_features = np.zeros((len(train_df), 3))
test_features = np.zeros((len(test_df), 3))

for fold, (train_idx, test_idx) in enumerate(skf):
    X_train = train_df.iloc[train_idx, :].reset_index(drop=True)
    
    X_test = train_df.iloc[test_idx, :].reset_index(drop=True)
    
    frequencies = {}
    frequencies_week_day = {}
    for vehicle in np.unique(X_train['vehicle'].values):
        filtered_df = X_train[X_train['vehicle'] == vehicle]
        frequencies[vehicle] = [len(filtered_df[filtered_df['occupancy'] == 'low'])/len(filtered_df),
                                len(filtered_df[filtered_df['occupancy'] == 'medium'])/len(filtered_df),
                                len(filtered_df[filtered_df['occupancy'] == 'high'])/len(filtered_df)]
        
    for i, idx in enumerate(test_idx):
        sample = X_test.iloc[i, :]
        if sample['vehicle'] in frequencies:
            train_features[idx, :] = frequencies[sample['vehicle']]
            
for vehicle in np.unique(train_df['vehicle'].values):
    filtered_df = train_df[train_df['vehicle'] == vehicle]
    frequencies[vehicle] = [len(filtered_df[filtered_df['occupancy'] == 'low'])/len(filtered_df),
                            len(filtered_df[filtered_df['occupancy'] == 'medium'])/len(filtered_df),
                            len(filtered_df[filtered_df['occupancy'] == 'high'])/len(filtered_df)]
    

for i in range(len(test_df)):
    sample = test_df.iloc[i, :]
    if sample['vehicle'] in frequencies:
        test_features[i, :] = frequencies[sample['vehicle']]
        
train_df = pd.DataFrame(np.hstack((train_df.as_matrix(), train_features)), 
                        columns=list(train_df.columns) + ['low_freq', 'med_freq', 'high_freq'])

test_df = pd.DataFrame(np.hstack((test_df.as_matrix(), test_features)), 
                       columns=list(test_df.columns) + ['low_freq', 'med_freq', 'high_freq'])

In [69]:
NR_FOLDS = 500
skf = StratifiedKFold(train_df['occupancy'].values, n_folds=10, shuffle=True, random_state=1337)

train_features = np.zeros((len(train_df), 3))
test_features = np.zeros((len(test_df), 3))

for fold, (train_idx, test_idx) in enumerate(skf):
    X_train = train_df.iloc[train_idx, :].reset_index(drop=True)
    
    X_test = train_df.iloc[test_idx, :].reset_index(drop=True)
    
    frequencies = {}
    frequencies_week_day = {}
    for vehicle in np.unique(X_train['week_day'].values):
        filtered_df = X_train[X_train['week_day'] == vehicle]
        frequencies[vehicle] = [len(filtered_df[filtered_df['occupancy'] == 'low'])/len(filtered_df),
                                len(filtered_df[filtered_df['occupancy'] == 'medium'])/len(filtered_df),
                                len(filtered_df[filtered_df['occupancy'] == 'high'])/len(filtered_df)]
        
    for i, idx in enumerate(test_idx):
        sample = X_test.iloc[i, :]
        if sample['week_day'] in frequencies:
            train_features[idx, :] = frequencies[sample['week_day']]
            
for vehicle in np.unique(train_df['week_day'].values):
    filtered_df = train_df[train_df['week_day'] == vehicle]
    frequencies[vehicle] = [len(filtered_df[filtered_df['occupancy'] == 'low'])/len(filtered_df),
                            len(filtered_df[filtered_df['occupancy'] == 'medium'])/len(filtered_df),
                            len(filtered_df[filtered_df['occupancy'] == 'high'])/len(filtered_df)]
    

for i in range(len(test_df)):
    sample = test_df.iloc[i, :]
    if sample['week_day'] in frequencies:
        test_features[i, :] = frequencies[sample['week_day']]
        
train_df = pd.DataFrame(np.hstack((train_df.as_matrix(), train_features)), 
                        columns=list(train_df.columns) + ['low_freq_wd', 'med_freq_wd', 'high_freq_wd'])

test_df = pd.DataFrame(np.hstack((test_df.as_matrix(), test_features)), 
                       columns=list(test_df.columns) + ['low_freq_wd', 'med_freq_wd', 'high_freq_wd'])

train_df = train_df.drop('week_day', axis=1)
test_df = test_df.drop('week_day', axis=1)

In [72]:
labels_df = train_df['occupancy'].map({'low': 0, 'medium': 1, 'high': 2})
features_df = train_df.drop(['occupancy' ,'id', 'day',
                             'vehicle', 'from'], axis=1)
test_features_df = test_df.drop(['occupancy' ,'id', 'day',
                                 'vehicle', 'from'], axis=1)

features_df = features_df.convert_objects(convert_numeric=True)
test_features_df = test_features_df.convert_objects(convert_numeric=True)

NR_FEATURES = 25
NR_FOLDS = 5

skf = StratifiedKFold(labels_df.values, n_folds=NR_FOLDS, shuffle=True, random_state=1337)


accuracies = []
for fold, (train_idx, test_idx) in enumerate(skf):
    print ('Fold', fold+1, '/', NR_FOLDS)
    X = features_df.iloc[train_idx, :].reset_index(drop=True)
    y = labels_df.iloc[train_idx].reset_index(drop=True)
    
    X_test = features_df.iloc[test_idx, :].reset_index(drop=True)
    y_test = labels_df.iloc[test_idx].reset_index(drop=True)
    
    X, y, X_test = extract_distance_features(X, y, X_test, 
                                             n_neighbors=[2**i for i in range(4)],
                                             _type='normal')
    
    msk = np.random.rand(len(X)) < 0.9
    X_train = X[msk]
    X_val = X[~msk]
    y_train = y[msk]
    y_val = y[~msk]
    
    weights = np.zeros((len(y_train)))
    for i in range(len(y_train)):
        label = y_train.iloc[i]
#         weights[i] = [1, 1.5, 1.25][int(label)]
        weights[i] = 1
        
    xgb = XGBClassifier(n_estimators=25000, max_depth=5, min_child_weight=4, learning_rate=0.0001,
                        colsample_bytree=0.5, subsample=0.66, gamma=0., nthread=-1, objective='multi:softmax')
    xgb.fit(X_train, y_train, sample_weight=weights, verbose=100,
            eval_metric=['merror', 'mlogloss'], eval_set=[(X_train, y_train), (X_val, y_val)], 
            early_stopping_rounds=50)

    results = xgb.evals_result()
    epochs = len(results['validation_0']['merror'])
    x_axis = range(0, epochs)
    fig, ax = plt.subplots()
    ax.plot(x_axis, results['validation_0']['mlogloss'], label='logloss Train')
    ax.plot(x_axis, results['validation_1']['mlogloss'], label='logloss Val')
    ax.plot(x_axis, results['validation_0']['merror'], label='error Train')
    ax.plot(x_axis, results['validation_1']['merror'], label='error Val')
    ax.legend(loc='center left')
    plt.ylabel('Error')
    plt.title('XGBoost Logloss/Classification Error')
    plt.show()
    
    selected_features_idx = xgb.feature_importances_.argsort()[-NR_FEATURES:][::-1]
    plt.bar(range(len(selected_features_idx)), [xgb.feature_importances_[i] for i in selected_features_idx])
    plt.xticks(range(len(selected_features_idx)), [X.columns[i] for i in selected_features_idx], rotation='vertical')
    plt.show()

    predictions = xgb.predict(X_test)
    conf_matrix = confusion_matrix(y_test, predictions)
    print(conf_matrix)
    accuracy = sum([conf_matrix[i][i] for i in range(len(conf_matrix))])/np.sum(conf_matrix)
    print('accuracy:', accuracy)
    accuracies.append(accuracy)
    
print('Avg accuracy:', np.mean(accuracies), np.std(accuracies))

Fold 1 / 5
[0]	validation_0-merror:0.416964	validation_0-mlogloss:1.09859	validation_1-merror:0.478261	validation_1-mlogloss:1.0986
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 50 rounds.
[100]	validation_0-merror:0.330007	validation_0-mlogloss:1.09574	validation_1-merror:0.496894	validation_1-mlogloss:1.09696
[200]	validation_0-merror:0.329294	validation_0-mlogloss:1.09291	validation_1-merror:0.490683	validation_1-mlogloss:1.09531
[300]	validation_0-merror:0.329294	validation_0-mlogloss:1.09009	validation_1-merror:0.509317	validation_1-mlogloss:1.0937
[400]	validation_0-merror:0.328582	validation_0-mlogloss:1.08724	validation_1-merror:0.503106	validation_1-mlogloss:1.0921
[500]	validation_0-merror:0.329294	validation_0-mlogloss:1.08449	validation_1-merror:0.503106	validation_1-mlogloss:1.09054
[600]	validation_0-merror:0.327869	validation_0-mlogloss:1.08175	validation_1-merro

KeyboardInterrupt: 

In [75]:
labels_df = train_df['occupancy'].map({'low': 0, 'medium': 1, 'high': 2})
features_df = train_df.drop(['occupancy' ,'id', 'day',
                             'vehicle', 'from'], axis=1)
test_features_df = test_df.drop(['occupancy' ,'id', 'day',
                                 'vehicle', 'from'], axis=1)
print(len(list(features_df.columns)), len(set(features_df.columns)))
features_df, labels_df, test_features_df = extract_distance_features(features_df, labels_df, test_features_df, 
                                                                     n_neighbors=[2**i for i in range(4)],
                                                                     _type='normal')
features_df = features_df.convert_objects(convert_numeric=True)
test_features_df = test_features_df.convert_objects(convert_numeric=True)

xgb = XGBClassifier(n_estimators=10000, max_depth=5, min_child_weight=1, learning_rate=0.0001,
                    colsample_bytree=0.5, subsample=0.66, gamma=0., nthread=-1, objective='multi:softmax')

weights = np.zeros((len(labels_df)))
for i in range(len(labels_df)):
    label = labels_df.iloc[i]
    #weights[i] = [1, 1.1, 1.25][int(label)]
    weights[i] = 1
        
xgb.fit(features_df, labels_df, sample_weight=weights)

prediction_vectors = []
for prediction, _id in zip(xgb.predict(test_features_df), test_df['id'].values):
    prediction_vectors.append([_id, prediction])
prediction_df = pd.DataFrame(prediction_vectors)
prediction_df.columns = ['id', 'occupancy']
prediction_df.to_csv('submission.csv', index=False)

43 43


In [30]:
def extract_distance_features(X_train, y_train, X_test, n_neighbors=[4], _type='normal',
                              n_components=3):
    """
    X_train: training feature vectors (pandas Dataframe)
    y_train: corresponding labels of the training feature vectors (pandas Dataframe)
    X_test: the test feature vectors (pandas Dataframe)
    n_neighbors: a list where each elements indicates how many neighbors have to be taken into account
                 the number of features returned is equal to the cardina,lity of this list
    _type: 'normal' will use neighbors with least distance using the feature vectors; 'pca' will
           first apply Prinicipal Component Analysis
    """
    
    _classes = np.unique(y_train.values)
    
    # Create the new column names
    _columns = list(X_train.columns)
    for _class in _classes:
        for n_neighbor in n_neighbors:
            _columns.append('distance_'+_type+'_'+str(n_neighbor)+'-'+str(_class))
    
    X_train_matrix = X_train.as_matrix()
    y_train_matrix = y_train.as_matrix()
    X_test_matrix = X_test.as_matrix()
    
    if _type == 'pca':
        X_train = X_train.fillna(0)
        X_test = X_test.fillna(0)
        pca = PCA(n_components=n_components)
        X_train = pd.DataFrame(pca.fit_transform(X_train, y_train))
        X_test = pd.DataFrame(pca.fit_transform(X_test))
    
    # First, we initialize the KDTrees
    trees = [cKDTree(X_train[y_train.values == _class]) for _class in _classes]
    
    # Calculate the distance features
    distance_features = np.zeros((len(X_train)+len(X_test), len(n_neighbors)*len(trees)))
    data = pd.concat([X_train, X_test])
    for i in range(len(data)):
        vector = data.iloc[i, :]
        k = 0
        for tree in trees:
            for n_neighbor in n_neighbors:
                distances, indices = tree.query(vector, k=n_neighbor+1, p=2)
                distance_features[i][k] = sum(distances[(distances[0] == 0):n_neighbor+(distances[0] == 0)])
                k += 1
    
    # Return X_train, y_train and X_test with new features (and their new columns)
    X_train = pd.DataFrame(np.hstack((X_train_matrix, distance_features[:len(X_train)])))
    X_train.columns = _columns
    X_test = pd.DataFrame(np.hstack((X_test_matrix, distance_features[len(X_train):])))
    X_test.columns = _columns
    return X_train, y_train, X_test

In [84]:
features_df, labels_df, test_features_df = extract_distance_features(features_df, labels_df, test_features_df, 
                                                                     n_neighbors=[2**i for i in range(1,5)])